In [1]:
import pandas as pd
import psycopg2
import pyodbc
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv


In [2]:
load_dotenv() #Necessário para carregar as variáveis presente no arquivo .env

# Credenciais do SQL Server

driver = "ODBC Driver 18 for SQL Server"
server = 'localhost'
database = 'AdventureWorks'
uid = os.getenv("sql_uid")
pwd = os.getenv("sql_pwd")

# Credenciais do Postgres

pg_server = 'localhost'
pg_database = 'adventureworks'
port = '5431' #A porta padrão é 5432, mas ao subir o meu banco num container docker, eu escolhi a 5431
pg_user = os.getenv("pg_user")
pg_pwd = os.getenv("pg_pwd")


In [3]:
# Montando as strings de conexões

# SQL Server

conn_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={uid};PWD={pwd};TrustServerCertificate=yes'
conn_scr = pyodbc.connect(conn_string)

# Postgres

engine = create_engine(f'postgresql://{pg_user}:{pg_pwd}@{pg_server}:{port}/{pg_database}')


### Carga de dados no banco de destino(Postgres)

In [4]:
fonte = pd.read_sql_query("""
SELECT
    cod_cliente,
    nome_completo,
    tipo_cliente
FROM Sales.clientes
WHERE cod_cliente <= 6
""", conn_scr)
fonte

/var/folders/85/7g55hdxn655gfsc90k47xqkr0000gn/T/ipykernel_30988/4037460507.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fonte = pd.read_sql_query("""


,cod_cliente,nome_completo,tipo_cliente
0,1,Ana Souza,Cliente Varejo
1,2,Bruno Oliveira,Cliente Comercial
2,3,Carlos Mendes,Cliente Varejo
3,4,Daniela Castro,Cliente Comercial
4,5,Eduardo Lima,Cliente Varejo
5,6,Fernanda Martins,Cliente Comercial


In [5]:
tabela = 'clientes'
fonte.to_sql(tabela,engine,if_exists='replace',index=False)

6

### Merge da tabela de origem com a de destino para a carga de dados novos

In [6]:
origem = pd.read_sql_query("""
SELECT
    cod_cliente,
    nome_completo,
    tipo_cliente
FROM Sales.clientes
""", conn_scr)
origem

/var/folders/85/7g55hdxn655gfsc90k47xqkr0000gn/T/ipykernel_30988/2338578742.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  origem = pd.read_sql_query("""


,cod_cliente,nome_completo,tipo_cliente
0,1,Ana Souza,Cliente Varejo
1,2,Bruno Oliveira,Cliente Comercial
2,3,Carlos Mendes,Cliente Varejo
3,4,Daniela Castro,Cliente Comercial
4,5,Eduardo Lima,Cliente Varejo
5,6,Fernanda Martins,Cliente Comercial
6,7,Gabriel Silva,Cliente Varejo
7,8,Helena Costa,Cliente Comercial
8,9,Igor Santos,Cliente Varejo
9,10,Juliana Freitas,Cliente Comercial


In [7]:
destino = pd.read_sql_query("""
SELECT
    cod_cliente
FROM clientes
""", engine)
destino

,cod_cliente
0,1
1,2
2,3
3,4
4,5
5,6


In [8]:
tbl_merge = origem.merge(destino,
                        how = 'left',
                        on = 'cod_cliente',
                        indicator=True)
tbl_merge

,cod_cliente,nome_completo,tipo_cliente,_merge
0,1,Ana Souza,Cliente Varejo,both
1,2,Bruno Oliveira,Cliente Comercial,both
2,3,Carlos Mendes,Cliente Varejo,both
3,4,Daniela Castro,Cliente Comercial,both
4,5,Eduardo Lima,Cliente Varejo,both
5,6,Fernanda Martins,Cliente Comercial,both
6,7,Gabriel Silva,Cliente Varejo,left_only
7,8,Helena Costa,Cliente Comercial,left_only
8,9,Igor Santos,Cliente Varejo,left_only
9,10,Juliana Freitas,Cliente Comercial,left_only


In [9]:
dados_novos = tbl_merge[tbl_merge["_merge"] == "left_only"][["cod_cliente", "nome_completo", "tipo_cliente"]]
dados_novos


,cod_cliente,nome_completo,tipo_cliente
6,7,Gabriel Silva,Cliente Varejo
7,8,Helena Costa,Cliente Comercial
8,9,Igor Santos,Cliente Varejo
9,10,Juliana Freitas,Cliente Comercial


In [10]:
tabela = 'clientes'
dados_novos.to_sql(tabela,engine,if_exists='append',index=False)

4